In [1]:
import subprocess
import sys
import os

# 📌 Liste des bibliothèques nécessaires
REQUIRED_PACKAGES = [
    "torch", "torchaudio", "torchaudio", "whisper", "noisereduce",
    "pydub", "soundfile", "librosa", "webrtcvad", "ffmpeg-python",
    "silero-vad", "pyannote.audio"
]

# 📌 Fonction pour installer les paquets pip
def install_packages():
    print("\n📦 Installation des dépendances...\n")
    for package in REQUIRED_PACKAGES:
        subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)

# 📌 Fonction pour vérifier et installer FFmpeg si absent
def install_ffmpeg():
    print("\n🔍 Vérification de FFmpeg...\n")
    
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        print("✅ FFmpeg est déjà installé.")
    except FileNotFoundError:
        print("⚠️ FFmpeg non trouvé ! Installation en cours...\n")
        
        if sys.platform == "win32":
            subprocess.run(["choco", "install", "ffmpeg", "-y"], shell=True)
        elif sys.platform == "darwin":
            subprocess.run(["brew", "install", "ffmpeg"], shell=True)
        else:  # Linux
            subprocess.run(["sudo", "apt", "install", "ffmpeg", "-y"], shell=True)

# 📌 Exécution des installations
if __name__ == "__main__":
    install_packages()
    install_ffmpeg()
    print("\n✅ Installation terminée avec succès ! 🚀")


from pydub import AudioSegment
import subprocess
import noisereduce as nr
import librosa
import soundfile as sf

import torch
import torchaudio
from silero_vad import get_speech_timestamps, collect_chunks
import io
import numpy as np

from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.core import Segment
from tqdm import tqdm
import whisper




📦 Installation des dépendances...

  Using cached whisper-1.1.10-py3-none-any.whl

🔍 Vérification de FFmpeg...

✅ FFmpeg est déjà installé.

✅ Installation terminée avec succès ! 🚀


/opt/anaconda3/envs/Deep_Vision/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


# Function

In [ ]:
import subprocess

def extract_audio(video_path, output_audio_path):
    '''
    Explication des options ffmpeg
    -ac 1 → Convertit l’audio en mono
    -ar 16000 → Définit la fréquence d’échantillonnage à 16 kHz (utile pour certaines applications)
    -q:a 0 → Qualité audio maximale
    -map a → Extrait uniquement la piste audio
    '''
    command = f'ffmpeg -i "{video_path}" -vn -ac 1 -ar 16000 -q:a 0 -map a "{output_audio_path}"'
    subprocess.run(command, shell=True)



## Load audio

In [ ]:


def extract_audio(video_path, output_audio_path):
    '''
    Explication des options ffmpeg
    -ac 1 → Convertit l’audio en mono
    -ar 16000 → Définit la fréquence d’échantillonnage à 16 kHz (utile pour certaines applications)
    -q:a 0 → Qualité audio maximale
    -map a → Extrait uniquement la piste audio
    '''
    command = f'ffmpeg -i "{video_path}" -vn -ac 1 -ar 16000 -q:a 0 -map a "{output_audio_path}"'
    subprocess.run(command, shell=True)



## Snippet

In [ ]:


def time_to_seconds(time_str):
    """Convertit une chaîne de temps HH:MM:SS en secondes."""
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

def extract_snippets(audio_path, output_path, snippets):
    """
    Extrait et concatène des parties spécifiques d'un fichier audio.

    :param audio_path: Chemin du fichier audio d'entrée
    :param output_path: Chemin du fichier audio de sortie
    :param snippets: Liste de listes [["HH:MM:SS", "HH:MM:SS"]]
    """
    audio = AudioSegment.from_file(audio_path)
    extracted_audio = AudioSegment.empty()  # Audio final vide au départ

    for start, end in snippets:
        start_sec = time_to_seconds(start)
        end_sec = time_to_seconds(end)
        extracted_audio += audio[start_sec * 1000:end_sec * 1000]  # Convertir en millisecondes

    # Sauvegarde du fichier final
    extracted_audio.export(output_path, format="wav")



## Noise 
📌 Outils : noisereduce, pydub, DeepFilterNet
📌 Objectif : Supprimer le bruit, équilibrer le volume.

In [3]:


def clean_audio(input_audio, output_audio):
    y, sr = librosa.load(input_audio, sr=16000)
    reduced_noise = nr.reduce_noise(y=y, sr=sr)
    
    # Sauvegarde du fichier nettoyé
    sf.write(output_audio, reduced_noise, sr)

    # Normalisation du volume
    sound = AudioSegment.from_file(output_audio)
    normalized_sound = sound.apply_gain(-sound.dBFS)
    normalized_sound.export(output_audio, format="wav")



/opt/anaconda3/envs/Deep_Vision/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/Deep_Vision/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/Deep_Vision/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

## Filtrage pour ne garder que la voix (VAD - Voice Activity Detection) 🔊

📌 Outil : Silero VAD (plus précis que WebRTC)
📌 Objectif : Supprimer les parties sans voix (silence, musique, bruit).

In [ ]:
import torchaudio
import webrtcvad
import struct
from pydub import AudioSegment

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Découpe l'audio en trames de frame_duration_ms millisecondes."""
    frame_size = int(sample_rate * (frame_duration_ms / 1000.0)) * 2  # 16-bit (2 bytes par échantillon)
    for i in range(0, len(audio), frame_size):
        yield audio[i:i + frame_size]

def extract_voice_webrtc(audio_file, output_file, aggressiveness=3):
    """
    Extrait uniquement les parties parlées d'un fichier audio en utilisant WebRTC VAD.
    
    :param audio_file: Chemin du fichier audio d'entrée (WAV uniquement)
    :param output_file: Chemin du fichier audio de sortie
    :param aggressiveness: Niveau d'agressivité du VAD (0 = le plus permissif, 3 = le plus strict)
    """
    # Charger l'audio
    waveform, sr = torchaudio.load(audio_file)
    audio = waveform.numpy().flatten()

    # Convertir en mono si nécessaire
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Normaliser l'audio en 16-bit PCM
    audio_int16 = (audio * 32767).astype("int16")
    audio_bytes = struct.pack('<' + ('h' * len(audio_int16)), *audio_int16)

    # Initialiser WebRTC VAD
    vad = webrtcvad.Vad(aggressiveness)
    frame_duration_ms = 30  # Taille des trames (10, 20 ou 30 ms recommandées)
    frames = list(frame_generator(frame_duration_ms, audio_bytes, sr))

    # Détecter les parties parlées
    speech_frames = [frame for frame in frames if vad.is_speech(frame, sr)]

    if not speech_frames:
        print("❌ Aucune voix détectée.")
        return

    # Fusionner les segments parlés
    speech_audio = b"".join(speech_frames)

    # Sauvegarde du fichier final
    sound = AudioSegment(data=speech_audio, sample_width=2, frame_rate=sr, channels=1)
    sound.export(output_file, format="wav")



Transcription de la voix en texte 📝

📌 Outil : Whisper (OpenAI)
📌 Objectif : Transformer l’audio en texte fiable.

In [ ]:

"""
def transcribe_audio_to_text(audio_path, output_text_path):
    model = whisper.load_model("large")  # Choisir "base", "medium", "large" selon GPU dispo
    result = model.transcribe(audio_path)
    
    with open(output_text_path, "w") as text_file:
        text_file.write(result["text"])
    
    return output_text_path

""""


In [18]:
import wave
import json
from vosk import Model, KaldiRecognizer

def transcribe_audio_vosk(audio_path, output_text_path, model_path="/Users/lorisfabbro/Desktop/Divers/MA2/Deep learning/Project/Deep_learning/vosk-model-small-en-us-0.15"):
    """
    Transcrit un fichier audio et enregistre le texte dans un fichier.
    
    :param audio_path: Chemin du fichier audio d'entrée
    :param output_text_path: Chemin du fichier texte de sortie
    :param model_path: Chemin du modèle Vosk (par défaut, modèle anglais léger)
    """
    model = Model(model_path)
    recognizer = KaldiRecognizer(model, 16000)

    with wave.open(audio_path, "rb") as audio_file:
        text = ""

        while True:
            data = audio_file.readframes(4000)  # Lire par petits blocs pour meilleure précision
            if len(data) == 0:
                break

            if recognizer.AcceptWaveform(data):
                result = json.loads(recognizer.Result())
                text += result["text"] + " "
            else:
                result = json.loads(recognizer.PartialResult())
                text += result["partial"] + " "

        # Récupérer le dernier bloc de texte
        final_result = json.loads(recognizer.FinalResult())
        text += final_result["text"]

        # Écrire la transcription dans un fichier texte
        with open(output_text_path, "w", encoding="utf-8") as text_file:
            text_file.write(text)

        print(f"✅ Transcription enregistrée dans : {output_text_path}")



# CODE 

In [5]:

Video_path = "path/hate_video_18.mp4" #2.2 Mo
label = "Hate"
snippet = [['00:00:10', '00:00:20'], ['00:00:23', '00:00:33']]
target = "Blacks"
output_audio_path = "path/final_audio.wav" #1.7 Mo --> 

In [4]:
extract_audio(Video_path, output_audio_path)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --

In [6]:
extract_snippets(output_audio_path, output_audio_path, snippet)

In [9]:
clean_audio(output_audio_path, "path/final_audio_clean.wav")

In [3]:
print(extract_voice)  # Devrait afficher <function extract_voice at 0x...>


<function extract_voice at 0x7fa0c62938b0>


In [2]:

# Exemple d'utilisation
extract_voice_webrtc("path/final_audio_clean.wav", "path/final_audio_clean_voice.wav")

✅ Audio extrait et enregistré sous path/final_audio_clean_voice.wav


In [11]:

transcribe_audio_google("path/final_audio_clean_voice.wav", "path/final_audio_clean_voice.txt")

❌ Erreur : Impossible de reconnaître l'audio (silence ou mauvaise qualité).


In [19]:


transcribe_audio_vosk("path/final_audio_clean_voice.wav" , "path/final_audio_clean_voice_vosk.txt")


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/lorisfabbro/Desktop/Divers/MA2/Deep learning/Project/Deep_learning/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /Users/lorisfabbro/Desktop/Divers/MA2/Deep learning/Project/Deep_learning/vosk-model-small-en-us-0.15/graph/HCLr.fst /Users/lorisfabbro/Desktop/Divers/MA2/Deep learning/Project/Deep_learning/vosk-model-small-en-us-0.15/graph/G

✅ Transcription enregistrée dans : path/final_audio_clean_voice_vosk.txt
